In [1]:
# Prompt from ...
#
from dataclasses import dataclass

In [2]:
from parsita import TextParsers, lit, reg

In [3]:
from minichain import Backend, JinjaPrompt, SimplePrompt, start_chain

In [4]:
# Define the states of the bot.
@dataclass
class IntermediateState:
    s: str
@dataclass
class FinalState:
    s: str
States = FinalState | IntermediateState

In [5]:
class SelfAsk(JinjaPrompt[States]):
    template_file = "selfask.pmpt.tpl"
    stop_template = "\nIntermediate answer:"

    class SelfAskParser(TextParsers):  # type: ignore
        follow = (lit("Follow up:") >> reg(r".*")) > IntermediateState
        finish = (lit("So the final answer is: ") >> reg(r".*")) > FinalState
        response = follow | finish

    def parse(self, inp: str) -> States:
        return self.SelfAskParser.response.parse(inp).or_die()  # type: ignore

In [6]:
SelfAsk().show({"input": "What is the zip code of the city where George Washington was born?",
                "agent_scratchpad":True},
               "Follow up: Where was George Washington born?")

HTML(html='\n<!-- <link rel="stylesheet" href="https://cdn.rawgit.com/Chalarangelo/mini.css/v3.0.1/dist/mini-default.min.css"> -->\n <main class="container">\n\n<h3>SelfAsk</h3>\n\n<dl>\n  <dt>Input:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="p">{</span><span class="s1">&#39;input&#39;</span><span class="p">:</span> <span class="s1">&#39;What is the zip code of the city where George Washington was born?&#39;</span><span class="p">,</span> <span class="s1">&#39;agent_scratchpad&#39;</span><span class="p">:</span> <span class="kc">True</span><span class="p">}</span>\n</pre></div>\n\n\n  </dd>\n\n  <dt> Full Prompt: </dt>\n  <dd>\n    <details>\n      <summary></summary>\n      <p>Question: Who lived longer, Muhammad Ali or Alan Turing?<br>Are follow up questions needed here: Yes.<br>Follow up: How old was Muhammad Ali when he died?<br>Intermediate answer: Muhammad Ali was 74 years old when he died.<br>Follow up: How old was Alan Turing when he died?<br>Intermediate answer: Alan Turing was 41 years old when he died.<br>So the final answer is: Muhammad Ali<br><br>Question: When was the founder of craigslist born?<br>Are follow up questions needed here: Yes.<br>Follow up: Who was the founder of craigslist?<br>Intermediate answer: Craigslist was founded by Craig Newmark.<br>Follow up: When was Craig Newmark born?<br>Intermediate answer: Craig Newmark was born on December 6, 1952.<br>So the final answer is: December 6, 1952<br><br>Question: Who was the maternal grandfather of George Washington?<br>Are follow up questions needed here: Yes.<br>Follow up: Who was the mother of George Washington?<br>Intermediate answer: The mother of George Washington was Mary Ball Washington.<br>Follow up: Who was the father of Mary Ball Washington?<br>Intermediate answer: The father of Mary Ball Washington was Joseph Ball.<br>So the final answer is: Joseph Ball<br><br>Question: Are both the directors of Jaws and Casino Royale from the same country?<br>Are follow up questions needed here: Yes.<br>Follow up: Who is the director of Jaws?<br>Intermediate answer: The director of Jaws is Steven Spielberg.<br>Follow up: Where is Steven Spielberg from?<br>Intermediate answer: The United States.<br>Follow up: Who is the director of Casino Royale?<br>Intermediate answer: The director of Casino Royale is Martin Campbell.<br>Follow up: Where is Martin Campbell from?<br>Intermediate answer: New Zealand.<br>So the final answer is: No<br><br>Question: <div style=\'color:red\'>What is the zip code of the city where George Washington was born?</div><br>Are followup questions needed here: Yes.<br></p>\n    </details>\n  </dd>\n  \n  <dt> Response: </dt>\n  <dd>     \n    Follow up: Where was George Washington born?\n  </dd>\n\n  <dt>Value:</dt>\n  <dd>\n<div class="highlight"><pre><span></span><span class="n">IntermediateState</span><span class="p">(</span><span class="n">s</span><span class="o">=</span><span class="s1">&#39;Where was George Washington born?&#39;</span><span class="p">)</span>\n</pre></div>\n\n  </dd>\n</main>\n')

In [7]:
def selfask(inp: str, openai: Backend, google: Backend) -> str:
    prompt1 = SelfAsk(openai)
    prompt2 = SimplePrompt(google)
    suffix = ""
    for i in range(3):
        r1 = prompt1(
            dict(input=inp, suffix=suffix, agent_scratchpad=True), name=f"Chat {i}"
        )
        if isinstance(r1.val, FinalState):
            break
        out = prompt2(r1.val.s, name=f"Google{i}")
        suffix += "\nIntermediate answer:" + out.echo
    return r1.val.s

In [8]:
with start_chain("selfask") as backend:
    result = selfask(
        "What is the zip code of the city where George Washington was born?",
        backend.Mock(
            [
                "Follow up: Where was George Washington born?",
                "Follow up: What is the zip code of Virginia?",
                "So the final answer is: 12312",
            ]
        ),
        # OpenAI("")
        backend.Mock(["Virginia", "12312"]),
    )
print(result)

12312


In [9]:
!eliot-tree -l 0 selfask.log | grep -v "succeeded" | grep -v "started"
    
    # Google("593a073fa4c730efe918e592a538b36e80841bc8f8dd4070c1566920f75ba140")))

7ddbd0c1-8097-4462-be99-111df5d2d298

64d9f8ad-084a-4216-987a-4c74e4162cb8
    │   ├── input: What is the zip code of Virginia?
    │   ├── prompt: What is the zip code of Virginia?
    │   ├── result: 12312
    │   │   └── returned: 12312

e9d4f97d-0b3f-4303-9c98-d904c1aa8f24
    │   ├── input: 
    │   │   ├── agent_scratchpad: True
    │   │   ├── input: What is the zip code of the city where George Washington was born?
    │   │   └── suffix: ⏎
    │   │       Intermediate answer:Where was George Washington born?Virginia⏎
    │   │       Intermediate answer:What is the zip code of Virginia?12312
    │   ├── prompt: Question: Who lived longer, Muhammad Ali or Alan Turing?⏎
    │   │   Are follow up questions needed here: Yes.⏎
    │   │   Follow up: How old was Muhammad Ali when he died?⏎
    │   │   Intermediate answer: Muhammad Ali was 74 years old when he died.⏎
    │   │   Follow up: How old was Alan Turing when he died?⏎
    │   │   Intermediate answer: Alan Turing was 41 years

    │   ├── input: Where was George Washington born?
    │   ├── prompt: Where was George Washington born?
    │   ├── result: Virginia
    │   │   └── returned: Virginia

